In [1]:
#| default_exp py2pyi

# Create delegated pyi

## Setup

In [2]:
from cosette import *
from aimagic.oai import *

In [3]:
create_magic(models[0])

In [4]:
%ai reset

In [5]:
#| export
import ast, sys, inspect, re, os, importlib.util, importlib.machinery

from ast import parse, unparse
from inspect import signature, getsource
from fastcore.utils import *
from fastcore.meta import delegates

## Basics

In [6]:
%%aip 0
Create a function to import dynamically the module referenced in `fn`

In [7]:
#| export
def imp_mod(module_path, package=None):
    module_path = str(module_path)
    module_name = os.path.splitext(os.path.basename(module_path))[0]
    spec = importlib.machinery.ModuleSpec(module_name, None, origin=module_path)
    module = importlib.util.module_from_spec(spec)
    spec.loader = importlib.machinery.SourceFileLoader(module_name, module_path)
    if package is not None: module.__package__ = package
    module.__file__ = os.path.abspath(module_path)
    spec.loader.exec_module(module)
    return module

In [8]:
fn = Path('test_py2pyi.py')

In [9]:
mod = imp_mod(fn)
a = mod.A()
a.h()

1

In [10]:
#| export
def _get_tree(mod):
    return parse(getsource(mod))

In [11]:
tree = _get_tree(mod)

In [12]:
#| export
@patch
def __repr__(self:ast.AST):
    return unparse(self)

@patch
def _repr_markdown_(self:ast.AST):
    return f"""```python
{self!r}
```"""

In [13]:
# for o in enumerate(tree.body): print(o)

In [14]:
node = tree.body[4]
node

```python
def f(a: int, b: str='a') -> str:
    """I am f"""
    return 1
```

In [15]:
functypes = (ast.FunctionDef,ast.AsyncFunctionDef)

In [16]:
isinstance(node, functypes)

True

In [17]:
%%aip 0
How do I check if a function node `node` has a decorator named `name`?

In [18]:
#| export
def _deco_id(d:ast.Name|ast.Attribute)->bool:
    "Get the id for AST node `d`"
    return d.id if isinstance(d, ast.Name) else d.func.id

def has_deco(node:ast.FunctionDef|ast.AsyncFunctionDef, name:str)->bool:
    return any(_deco_id(d)==name for d in getattr(node, 'decorator_list', []))

In [19]:
nm = 'delegates'
has_deco(node, nm)

False

In [20]:
node = tree.body[5]
node

```python
@delegates(f)
def g(c, d: X, **kwargs) -> str:
    """I am g"""
    return 2
```

In [21]:
has_deco(node, nm)

True

## Function processing

In [22]:
def _proc_body   (node, mod): print('_proc_body', type(node))
def _proc_func   (node, mod): print('_proc_func', type(node))
def _proc_class  (node, mod): print('_proc_class', type(node))
def _proc_patched(node, mod): print('_proc_patched', type(node))

In [23]:
#| export
def _get_proc(node):
    if isinstance(node, ast.ClassDef): return _proc_class
    if not isinstance(node, functypes): return None
    if not has_deco(node, 'delegates'): return _proc_body
    if has_deco(node, 'patch'): return _proc_patched
    return _proc_func

In [24]:
#| export
def _proc_tree(tree, mod):
    for node in tree.body:
        proc = _get_proc(node)
        if proc: proc(node, mod)

In [25]:
#| export
def _proc_mod(mod):
    tree = _get_tree(mod)
    _proc_tree(tree, mod)
    return tree

In [26]:
_proc_mod(mod);

_proc_class <class 'ast.ClassDef'>
_proc_body <class 'ast.FunctionDef'>
_proc_func <class 'ast.FunctionDef'>
_proc_body <class 'ast.FunctionDef'>
_proc_class <class 'ast.ClassDef'>
_proc_class <class 'ast.ClassDef'>
_proc_patched <class 'ast.FunctionDef'>
_proc_patched <class 'ast.FunctionDef'>
_proc_body <class 'ast.FunctionDef'>


In [27]:
node.name

'g'

In [28]:
sym = getattr(mod, node.name)
sym

<function test_py2pyi.g(c, d: test_py2pyi.X, *, b: str = 'a') -> str>

In [29]:
sig = signature(sym)
print(sig)

(c, d: test_py2pyi.X, *, b: str = 'a') -> str


In [30]:
#| export
def sig2str(sig):
    s = str(sig)
    s = re.sub(r"<class '(.*?)'>", r'\1', s)
    s = re.sub(r"dynamic_module\.", "", s)
    return s

In [31]:
#| export
def ast_args(func):
    sig = signature(func)
    return ast.parse(f"def _{sig2str(sig)}: ...").body[0].args

In [32]:
newargs = ast_args(sym)
newargs

```python
c, d: test_py2pyi.X, *, b: str='a'
```

In [33]:
node.args

```python
c, d: X, **kwargs
```

In [34]:
node.args = newargs
node

```python
@delegates(f)
def g(c, d: test_py2pyi.X, *, b: str='a') -> str:
    """I am g"""
    return 2
```

In [35]:
#| export
def _body_ellip(n: ast.AST):
    stidx = 1 if isinstance(n.body[0], ast.Expr) and isinstance(n.body[0].value, ast.Str) else 0
    n.body[stidx:] = [ast.Expr(ast.Constant(...))]

In [36]:
_body_ellip(node)
node

```python
@delegates(f)
def g(c, d: test_py2pyi.X, *, b: str='a') -> str:
    """I am g"""
    ...
```

In [37]:
%%aip 0
Create a function to replace the parameter list of the source code of a function `f` with a different signature.
Also, it should replace the body of the function with just `pass`, and remove any decorators named 'delegates'.

In [38]:
#| export
def _update_func(node, sym):
    node.args = ast_args(sym)
    _body_ellip(node)
    node.decorator_list = [d for d in node.decorator_list if _deco_id(d) != 'delegates']

In [39]:
tree = _get_tree(mod)
node = tree.body[5]
node

```python
@delegates(f)
def g(c, d: X, **kwargs) -> str:
    """I am g"""
    return 2
```

In [40]:
_update_func(node, sym)
node

```python
def g(c, d: test_py2pyi.X, *, b: str='a') -> str:
    """I am g"""
    ...
```

In [41]:
#| export
def _proc_body(node, mod): _body_ellip(node)

In [42]:
#| export
def _proc_func(node, mod):
    sym = getattr(mod, node.name)
    _update_func(node, sym)

In [43]:
tree = _proc_mod(mod)
tree.body[5]

_proc_class <class 'ast.ClassDef'>
_proc_class <class 'ast.ClassDef'>
_proc_class <class 'ast.ClassDef'>
_proc_patched <class 'ast.FunctionDef'>
_proc_patched <class 'ast.FunctionDef'>


```python
def g(c, d: test_py2pyi.X, *, b: str='a') -> str:
    """I am g"""
    ...
```

## Patch

In [44]:
node = tree.body[9]
node

```python
@patch
@delegates(j)
def k(self: (A, B), b: bool=False, **kwargs):
    return 1
```

In [45]:
ann = node.args.args[0].annotation

In [46]:
if hasattr(ann, 'elts'): ann = ann.elts[0]

In [47]:
nm = ann.id
nm

'A'

In [48]:
cls = getattr(mod, nm)
sym = getattr(cls, node.name)

In [49]:
sig2str(signature(sym))

"(self: (test_py2pyi.A, test_py2pyi.B), b: bool = False, *, d: str = 'a')"

In [50]:
_update_func(node, sym)

In [51]:
node

```python
@patch
def k(self: (test_py2pyi.A, test_py2pyi.B), b: bool=False, *, d: str='a'):
    ...
```

In [52]:
#| export
def _proc_patched(node, mod):
    ann = node.args.args[0].annotation
    if hasattr(ann, 'elts'): ann = ann.elts[0]
    cls = getattr(mod, ann.id)
    sym = getattr(cls, node.name)
    _update_func(node, sym)

In [53]:
tree = _proc_mod(mod)
tree.body[9]

_proc_class <class 'ast.ClassDef'>
_proc_class <class 'ast.ClassDef'>
_proc_class <class 'ast.ClassDef'>


```python
@patch
def k(self: (test_py2pyi.A, test_py2pyi.B), b: bool=False, *, d: str='a'):
    ...
```

## Class and file

In [54]:
tree = _get_tree(mod)
node = tree.body[7]
node

```python
class A:

    @delegates(j)
    def h(self, b: bool=False, **kwargs):
        a = 1
        return a
```

In [55]:
node.body

[@delegates(j)
 def h(self, b: bool=False, **kwargs):
     a = 1
     return a]

In [56]:
#| export
def _proc_class(node, mod):
    cls = getattr(mod, node.name)
    _proc_tree(node, cls)

In [57]:
tree = _proc_mod(mod)
tree.body[7]

```python
class A:

    def h(self, b: bool=False, *, d: str='a'):
        ...
```

In [58]:
%%aip 0
Create a function that uses the functions above to create a new `pyi` file from a file `fn`, processed with `_proc_mod`

In [59]:
#| export
def create_pyi(fn, package=None):
    "Convert `fname.py` to `fname.pyi` by removing function bodies and expanding `delegates` kwargs"
    fn = Path(fn)
    mod = imp_mod(fn, package=package)
    tree = _proc_mod(mod)
    res = unparse(tree)
    fn.with_suffix('.pyi').write_text(res)

In [60]:
create_pyi(fn)

In [64]:
# fn = Path('/Users/jhoward/git/fastcore/fastcore/docments.py')
# create_pyi(fn, 'fastcore')

## Script

In [63]:
#| export
from fastcore.script import call_parse

In [64]:
#| export
@call_parse
def py2pyi(fname:str,  # The file name to convert
           package:str|None=None  # The parent package
          ):
    "Convert `fname.py` to `fname.pyi` by removing function bodies and expanding `delegates` kwargs"
    create_pyi(fname)

# Export -

In [65]:
#|hide
import nbdev; nbdev.nbdev_export()